In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np

# --- PENGATURAN ---
CHARACTER_IMAGE_PATH = './assets/denis.png' # Ganti dengan nama file gambar Anda
CHARACTER_SIZE = (80, 80) # Atur ukuran karakter (lebar, tinggi)
MOVE_SPEED = 7 # Kecepatan gerakan karakter
DEAD_ZONE_SIZE = 35 # Ukuran zona tengah
GAME_BG_COLOR = (0, 0, 0) # Warna background game (0,0,0) = Hitam
# --------------------

# Fungsi untuk overlay gambar (Sama seperti sebelumnya)
def overlay_image_alpha(background, overlay, x, y, size):
    """Menempelkan gambar 'overlay' ke 'background'."""
    try:
        overlay_resized = cv2.resize(overlay, size)
    except cv2.error as e:
        print(f"Error resizing overlay: {e}")
        return

    img_h, img_w, img_c = overlay_resized.shape
    bg_h, bg_w, _ = background.shape
    x_tl = x - img_w // 2
    y_tl = y - img_h // 2
    x1, x2 = max(0, x_tl), min(bg_w, x_tl + img_w)
    y1, y2 = max(0, y_tl), min(bg_h, y_tl + img_h)
    ix1, ix2 = max(0, -x_tl), min(img_w, bg_w - x_tl)
    iy1, iy2 = max(0, -y_tl), min(img_h, bg_h - y_tl)

    if x1 >= x2 or y1 >= y2 or ix1 >= ix2 or iy1 >= iy2:
        return

    roi = background[y1:y2, x1:x2]
    overlay_crop = overlay_resized[iy1:iy2, ix1:ix2]

    if roi.shape[0] != overlay_crop.shape[0] or roi.shape[1] != overlay_crop.shape[1]:
        overlay_crop = cv2.resize(overlay_crop, (roi.shape[1], roi.shape[0]))

    if img_c == 4:
        alpha_s = overlay_crop[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * overlay_crop[:, :, c] +
                            alpha_l * roi[:, :, c])
    else:
        background[y1:y2, x1:x2] = overlay_crop[:,:,:3]


# Muat gambar karakter (Sama seperti sebelumnya)
character_img = cv2.imread(CHARACTER_IMAGE_PATH, cv2.IMREAD_UNCHANGED)
if character_img is None:
    print(f"Error: Gagal memuat {CHARACTER_IMAGE_PATH}. Menggunakan lingkaran hijau.")
    character_img = np.zeros((100, 100, 4), dtype=np.uint8)
    cv2.circle(character_img, (50, 50), 40, (0, 255, 0, 255), -1)

# Inisialisasi MediaPipe & OpenCV (Sama seperti sebelumnya)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
success, temp_frame = cap.read()
if not success:
    print("Tidak bisa mengakses webcam.")
    exit()
height, width, _ = temp_frame.shape
temp_frame = None

# Posisi awal objek
obj_x, obj_y = width // 2, height // 2

# === Variabel untuk Pusat Referensi ===
initial_nose_x = None
initial_nose_y = None
center_set = False
# ============================================

print("Arahkan wajah ke tengah dan tunggu hingga garis silang muncul...")

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Gagal membaca frame.")
        continue

    # --- Buat Jendela Game (Layar Hitam Baru setiap frame) ---
    game_window = np.full((height, width, 3), GAME_BG_COLOR, dtype=np.uint8)
    # --------------------------------------------------------

    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    move_x = 0
    move_y = 0
    status_text = "CARI WAJAH..."

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            nose_tip = face_landmarks.landmark[1]
            current_nose_x = int(nose_tip.x * width)
            current_nose_y = int(nose_tip.y * height)

            if not center_set:
                initial_nose_x = current_nose_x
                initial_nose_y = current_nose_y
                center_set = True
                print(f"Pusat Referensi Diatur: ({initial_nose_x}, {initial_nose_y})")

            # --- Gambar di Jendela Kontrol (image) ---
            cv2.circle(image, (current_nose_x, current_nose_y), 5, (0, 0, 255), -1)

            if center_set:
                dead_zone_x1 = initial_nose_x - DEAD_ZONE_SIZE // 2
                dead_zone_x2 = initial_nose_x + DEAD_ZONE_SIZE // 2
                dead_zone_y1 = initial_nose_y - DEAD_ZONE_SIZE // 2
                dead_zone_y2 = initial_nose_y + DEAD_ZONE_SIZE // 2

                # Logika Gerakan (Sama seperti sebelumnya)
                if current_nose_y > dead_zone_y2:
                    move_y = MOVE_SPEED
                    move_x = 0
                    status_text = "BAWAH"
                elif current_nose_y < dead_zone_y1:
                    move_y = -MOVE_SPEED
                    move_x = 0
                    status_text = "ATAS"
                else:
                    if current_nose_x > dead_zone_x2:
                        move_x = MOVE_SPEED
                        move_y = 0
                        status_text = "KANAN"
                    elif current_nose_x < dead_zone_x1:
                        move_x = -MOVE_SPEED
                        move_y = 0
                        status_text = "KIRI"
                    else:
                        move_x = 0
                        move_y = 0
                        status_text = "BERHENTI"

                # Gambar Garis & Zona Mati di Jendela Kontrol
                cv2.line(image, (initial_nose_x, 0), (initial_nose_x, height), (0, 255, 255), 2)
                cv2.line(image, (0, initial_nose_y), (width, initial_nose_y), (0, 255, 255), 2)
                cv2.rectangle(image, (dead_zone_x1, dead_zone_y1),
                              (dead_zone_x2, dead_zone_y2), (255, 0, 0), 1)
            else:
                 status_text = "TUNGGU PUSAT DIATUR..."


    # Perbarui posisi objek & batasi
    obj_x += move_x
    obj_y += move_y
    char_w, char_h = CHARACTER_SIZE
    obj_x = max(char_w // 2, min(width - char_w // 2, obj_x))
    obj_y = max(char_h // 2, min(height - char_h // 2, obj_y))

    # --- Gambar Karakter di Jendela Game ---
    overlay_image_alpha(game_window, character_img, obj_x, obj_y, CHARACTER_SIZE)
    # -------------------------------------

    # Tampilkan status di Jendela Kontrol
    cv2.putText(image, f"Status: {status_text}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # --- Tampilkan KEDUA Jendela ---
    cv2.imshow('Controller', image)
    cv2.imshow('Game', game_window)
    # ---------------------------------

    # Hentikan program
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Arahkan wajah ke tengah dan tunggu hingga garis silang muncul...
Pusat Referensi Diatur: (288, 364)
